<a href="https://colab.research.google.com/github/AAStudioArt/Stable-Diffusion-Pickle-Scanner-GUI/blob/main/run_app_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@markdown # Install/Update Picklescan repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

if not os.path.exists("/content/gdrive/MyDrive/"):
  print('[1;31mGdrive not connected, using colab storage ...')
  time.sleep(4)
  !mkdir -p /content/gdrive/MyDrive/
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/MyDrive/sd
  %cd /content/gdrive/MyDrive/sd
  !$fgitclone -q --branch master https://github.com/AAStudioArt/Stable-Diffusion-Pickle-Scanner-GUI
  !mkdir -p /content/gdrive/MyDrive/sd/Stable-Diffusion-Pickle-Scanner-GUI
 
with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/Stable-Diffusion-Pickle-Scanner-GUI
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
print('[1;32m')
!git pull
clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Requirements

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/AAStudioArt/Stable-Diffusion-Pickle-Scanner-GUI/blob/main/requirements.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/MyDrive/sd/Stable-Diffusion-Pickle-Scanner-GUI'):
    !tar -C / --zstd -xf sd.tar.zst 
  !tar -C / --zstd -xf requirements.tar.zst
  !rm *.deb | rm *.zst | rm *.txt

pip install pysimplegui

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Model Download/Load
Files = "" #@param {type:"string"}
#@markdown - Insert the full path of the file who will be scanned

#@markdown Or
URL = "" #@param {type:"string"}
#@markdown - Insert the full path of the URL of item will be scanned


def main():

    #endregion window elements

    selected_input = {
        URL_RAD_KEY: {file_frame_elem: False, directory_frame_elem: False, url_frame_elem: True, huggingface_frame_elem: False},
        FILE_RAD_KEY: {file_frame_elem: True, directory_frame_elem: False, url_frame_elem: False, huggingface_frame_elem: False},
    }    

    flatten_ui_elements(window)

    def reset_ui():
        reset_files_state_display()
        update_status_state('')
        console_ml_elem.update(value='')
        scanned_files.clear()
        infected_files.clear()

    def update_status_state(value):
        status_state_txt_elem.update(value=value)
        status_state_txt_elem.Widget.config(background=color.GRAY_9900)  
        status_state_txt_elem.ParentRowFrame.config(background=color.GRAY_9900)

    def reset_files_state_display():
        status_info_txt_elem.update(value='Scanning. Please wait...') 
        scanned_files_txt_elem.update(value=SCANNED_FILES_DEF) 
        infected_files_txt_elem.update(value=INFECTED_FILES_DEF) 
        dangerous_globals_txt_elem.update(value=DANGEROUS_GLOBALS_DEF) 

    def scan():
        input_files = None
        if values[HUGGINGFACE_RAD_KEY]:
            input_files = values[HUGGINGFACE_INP_KEY]
            scan_target = scan_huggingface_model
        elif values[URL_RAD_KEY]:
            input_files = values[URL_INP_KEY]
            scan_target = scan_url
        elif values[DIRECTORY_RAD_KEY]:
            input_files = values[DIRECTORY_INP_KEY]
            scan_target = scan_directory_path
        elif values[FILE_RAD_KEY]:
            input_files = values[FILE_INP_KEY]
            scan_target = scan_file_path

        if input_files is None or len(input_files) <= 1:
            status_info_txt_elem.update(value='Nothing to scan')
            return

        try:
            return scan_target(input_files)
        except (RuntimeError, ConnectionRefusedError, AttributeError, FileNotFoundError,UnboundLocalError) as e:
            print(e.__class__.__name__, e)
            status_info_txt_elem.update(value='Fail to scan') 

    def update_results_display(scan_result):
        scanned_files_txt_elem.update(value=f'{SCANNED_FILES} {scan_result.scanned_files}') 
        infected_files_txt_elem.update(value=f'{INFECTED_FILES} {scan_result.infected_files}') 
        dangerous_globals_txt_elem.update(value=f'{DANGEROUS_GLOBALS} {scan_result.issues_count}') 
        scanned_files_len = len(scanned_files)
        infected_files_len = len(infected_files)

        print('')
        if scanned_files_len > 0:
            if infected_files_len == 0:
                status_state_txt_elem.update(text_color='white')
                status_state_txt_elem.Widget.config(background=color.DARK_GREEN)  
                status_state_txt_elem.ParentRowFrame.config(background=color.DARK_GREEN)
                status_state_txt_elem.update(value='PASS') 

        if infected_files_len > 0:
            print('')
            print('----------- INFECTED FILES -----------')
            print('')

            for file in infected_files:
                print(f"    {file}")

            status_state_txt_elem.update(text_color='white')
            status_state_txt_elem.Widget.config(background=color.RED_ORANGE)  
            status_state_txt_elem.ParentRowFrame.config(background=color.RED_ORANGE)
            status_state_txt_elem.update(value='FAIL') 

        status_info_txt_elem.update(value='Done') 

    while True:
        event, values = window.read()

        if event == sg.WIN_CLOSED:
            break
        
        if event == SCAN_BTN_KEY:
                reset_ui()
                scan_result = scan()

                if not isError:
                    if scan_result:
                        update_results_display(scan_result)

        if event == HUGGINGFACE_CLEAR_BTN_KEY:
            huggingface_inp_elem.update(value='')

        if event == URL_CLEAR_BTN_KEY:
            url_inp_elem.update(value='')

        if event in selected_input:
            for frame, visibility in selected_input[event].items():
                frame.update(visible=visibility)    

        if event in right_click_menu[1]:
            if values[HUGGINGFACE_RAD_KEY]:
                do_clipboard_operation(event, window, huggingface_inp_elem,STATUS_INFO_TXT_KEY)
            if values[URL_RAD_KEY]:
                do_clipboard_operation(event, window, url_inp_elem,STATUS_INFO_TXT_KEY)                
            if values[FILE_RAD_KEY]:
                do_clipboard_operation(event, window, file_inp_elem,STATUS_INFO_TXT_KEY)       
            if values[DIRECTORY_RAD_KEY]:
                do_clipboard_operation(event, window, directory_inp_elem,STATUS_INFO_TXT_KEY)   

            window[CONSOLE_ML_KEY].update(value='')
            
        support.buttons(event)

if __name__ == '__main__':
    main() 